<a href="https://colab.research.google.com/github/mateuszkaleta/pu-learning-example/blob/master/PU_Learning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przykład zastosowania metod PU Learning

Ten notebook zawiera praktyczny przykład do artykułu zamieszczonego na blogu technicznym ING (**link**).

## Dane

Na potrzeby prezentacji, wykorzystany zostanie dataset Banknote Authentication, dostępny tu: http://archive.ics.uci.edu/ml/datasets/banknote+authentication

Jest to zbiór danych o czterech atrybutach numerycznych, w zadaniu binarnej klasyfikacji. Zbiór ulegnie takiej modyfikacji, by odpowiadał warunkom PU learningu.

Atrybuty to wariancja, kurtoza, skośność oraz entropia zdjęć banknotów, poddanych transformacji falkowej.

## Algorytm

Jak wspomniane jest w artykule, zastosowana będzie tu metoda dwukrokowa. W pierwszym kroku należy utworzyć zbiór *reliable negatives*, w drugim zbudować klasyfikator binarny.

Jeżeli chodzi o implementację, to w pierwszym kroku dokonamy klasteryzacji zbioru. Zbiór zawierający najmniej znanych, pozytywnych instancji, posłuży jako zbiór negatywny w kroku budowy klasyfikatora binarnego. Taka metoda nazywa się C-CRNE (Clustering-based method for Collecting Reliable Negative Examples)

W drugim etapie, zostanie zbudowany model SVC.

In [0]:
!pip install scikit-learn==0.22.2 pandas==1.0.3 plotly==4.6.0


In [0]:
import sklearn
import pandas as pd
import numpy as np
import plotly.express as px

## Jak wyglądają dane?

In [0]:
features = ['variance', 'skewness', 'curtosis', 'entropy']
columns = features + ['class']
dataset_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
pu_dataset = pd.read_csv(dataset_url, names=columns)
pu_dataset['class'] = pd.Categorical(pu_dataset['class'])
pu_dataset.head(3)

In [0]:
pu_dataset.describe()

In [0]:
for i in range(len(features)):
  for j in range(i, len(features)):
    if i != j:
      fig = px.scatter(
          pu_dataset, 
          x=features[i], y=features[j], 
          color='class', 
          marginal_y="violin", 
          marginal_x="histogram", 
          title=f'{features[i]} against {features[j]}',
          color_discrete_sequence={1: 'rgba(20, 100, 180, 0.6)', 0: 'rgba(120, 100, 0, 0.6)'},
          width=1280, height=720
      )
      fig.show()
      

Dla celów porównawczych, przed usunięciem etykiet, pozostawiam zbiór walidacyjny. Na taki komfort można sobie pozwolić oczywiście jedynie w prezentacji.

In [0]:
# zapewnienie, że numpy będzie zwracać takie same liczby pseudolosowe
np.random.seed(3655867)

# losowe 20% instancji zostanie wybrane na zbiór walidacyjny
validation_set_share = 20/100

# na potrzeby prezentacji, wyrzucone zostanie 95% etykiet dla klasy pozytywnej i wszystkie dla klasy negatywnej
label_frequency = 5/100

all_indices = np.random.permutation(pu_dataset.index)
split_point = int(validation_set_share * len(pu_dataset))
training_indices = all_indices[split_point:]
test_indices = all_indices[:split_point]

class_distribution = pd.concat((pu_dataset.loc[training_indices]['class'], pu_dataset.loc[test_indices]['class']), axis=1).astype(float).describe().loc[['count', 'mean']]
class_distribution.columns = ['Training set', 'Test set']
class_distribution.index.name = 'Labels'
class_distribution

Mamy podobny rozkład klas w zbiorze testowym i treningowym, będziemy mogli zatem rzetelnie ocenić pracę klasyfikatora.

In [0]:
# usunięcie etykiet instancji negatywnych i części etykiet instancji pozytywnych (zgodnie z wartością label_frequency)
pu_dataset.loc[training_indices, 'selected'] = (np.random.uniform(low=0, high=1, size=len(training_indices)) < label_frequency).astype(int)
pu_dataset.loc[training_indices, 'selected'] *= pu_dataset.loc[training_indices, 'class'].astype(int)
pu_dataset['selected'] = pd.Categorical(pu_dataset['selected'])
pu_dataset.loc[training_indices, ['class', 'selected']].describe(include='all')

In [0]:
pu_dataset.loc[training_indices, 'class'] = None
pu_dataset.loc[test_indices, 'selected'] = None
pu_dataset.rename(columns={'selected': 'pu_label'}, inplace=True)
(pu_dataset['pu_label'].value_counts() / len(training_indices)).round(3)

In [0]:
pu_dataset.loc[test_indices, 'class'].value_counts()

In [0]:
pu_dataset.loc[training_indices, 'pu_label'].value_counts()

In [0]:
fig = px.scatter_matrix(
    pu_dataset.loc[training_indices],
    dimensions=features, 
    title='PU label distribution in training set',
    labels=pu_dataset.loc[training_indices, 'pu_label'], 
    color=pu_dataset.loc[training_indices, 'pu_label'], 
    color_discrete_sequence={1: 'rgb(20, 100, 180)', 0: 'rgba(180, 180, 180, 0.2)'}
)
fig.show()

fig = px.scatter_matrix(
    pu_dataset.loc[test_indices], 
    dimensions=features, 
    title='Clas distribution in test set',
    labels=pu_dataset.loc[test_indices, 'class'], 
    color=pu_dataset.loc[test_indices, 'class'], 
    color_discrete_sequence={1: 'rgba(20, 100, 180, 0.6)', 0: 'rgba(120, 100, 0, 0.6)'}
)
fig.show()

**Powyżej widać, że rozkład zmiennej zależnej jest zupełnie inny, niż w ogólnej populacji (tam ok. 45% to pozytywne przypadki).** 

# Budowa modelu PU Learning

Zastosowana zostanie metoda **C-CRNE** (*Clustering-based method for Collecting Reliable Negative Examples*) w połączeniu z **Iterative SVM**.



In [0]:
# standardyzacja danych
pu_dataset[features] = (pu_dataset[features].mean() - pu_dataset[features]) / pu_dataset[features].std()

assert ~(np.isin(test_indices, training_indices).any())
training_dataset = pu_dataset.loc[training_indices].copy()
test_dataset = pu_dataset.loc[test_indices].copy()

Pierwszym krokiem C-CRNE jest dokonanie klasteryzacji danych. W tym przypadku wybrałem KMeans, za pomocą silhouette_score wybieram optymalną liczbę klastrów.

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import itertools

# optymalizujemy liczbę klastrów
hyperparameters_space = {
    'n_clusters': list(range(3, 7)),
    'random_state': (23542, 31521, 9114, 4507)
}

best_clusterer = None
best_score = -1  # minimalna wartość silhouette score
best_parameters = None

run = 1
for parameters_set in list(itertools.product(*hyperparameters_space.values())):
  params = dict(zip(hyperparameters_space.keys(), parameters_set))
  # faktyczna klasteryzacja
  clusterer = KMeans(**params, n_jobs=4)
  cluster_labels = clusterer.fit_predict(training_dataset[features])
  # ewaluacja
  silhouette_mean = silhouette_score(training_dataset[features], cluster_labels)
  print(f'Run #{run}. Silhouette score: {silhouette_mean}')
  # wybierany jest najlepszy podział
  if silhouette_mean >= best_score:
    best_score = silhouette_mean
    best_clusterer = clusterer
    best_parameters = params
  run += 1

print(f'Best score: {best_score}. Best parameters: {str(best_parameters)}')

Następnie, spośród obliczonych klastrów, wybieramy ten, który zawiera najmniejszy odsetek znanych nam instancji klasy pozytywnej. Robimy to w myśl założenia o gładkości, czyli szukamy takiej grupy obserwacji, które mają najmniej wspólnego ze znanymi pozytywnymi przypadkami.

In [0]:
training_dataset['cluster'] = best_clusterer.predict(training_dataset[features])
positives_per_cluster = training_dataset.groupby('cluster')['pu_label']
positives_per_cluster_counts = positives_per_cluster.value_counts()
positives_per_cluster_share = (positives_per_cluster_counts / positives_per_cluster.count()).loc[:, 1].sort_values()
positives_per_cluster_share

Upewniamy się jeszcze, że wybrany klaster nie jest zbyt mały. Mogłoby się zdarzyć, że byłaby to nawet pojedyncza obserwacja, a ponieważ zaraz będziemy trenowali klasyfikator binarny, chcemy mieć sensownych rozmiarów próbkę instancji nieokreślonej klasy.

In [0]:
best_cluster = positives_per_cluster_share.iloc[:1]
100 * positives_per_cluster_counts.loc[best_cluster.index] / len(training_indices)

Jak widać, mamy całkiem przyjemny rezultat - aż czwarta część zbioru treningowego znalazła się w klastrze reliable negatives. Pozostaje wytrenowanie klasyfikatora.

In [0]:
reliable_negatives = training_dataset[training_dataset['cluster'] == best_cluster.index[0]].copy()
reliable_negatives = reliable_negatives[reliable_negatives['pu_label'] != 1]
positives = training_dataset[training_dataset['pu_label'] == 1]
positives = positives.loc[positives.index.isin(training_indices)]
len(reliable_negatives), len(positives), len(training_indices)

## Iterative SVM

Według tej techniki, kolejnym krokiem jest wytrenowanie modelu SVM na zbiorze znanych pozytywów i klastrze reliable negatives. Nie bez przypadku w nazwie metody występuje "iterative".

Po utworzeniu klasyfikatora w danej iteracji, dokonana zostanie predykcja na zbiorze nieoznaczonym (kilkaset obserwacji, które nie biorą udziału w treningu modelu). Te instancje, które model wskaże jako negatywne, poszerzą w kolejnej iteracji zbiór reliable negatives. Należy jeszcze zadać jakieś kryterium stopu, np. maksymalną ilość iteracji, czy brak zmian w rozkładzie predykcji.

Dodam, że będę tu wykonywał walidację krzyżową, chociaż należy pamiętać, że nie odpowiada to w pełni sytuacji klasyfikacji binarnej. Wciąż bowiem operujemy na zbiorze pozytywów i klastra, który *podejrzewamy* o zawieranie w większości negatywów.

In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import f1_score, make_scorer

# będziemy przechowywali najlepsze estymatory z kolejnych iteracji
best_estimators = []

# parametry do ewaluacji w każdej iteracji
params_grid = {
    'C': (.1, .3, .5, 1),
    'kernel': ('poly', 'rbf', 'sigmoid'),
    'degree': (3, 4),
    'random_state': (235325, 3462469)
}

last_iteration_unlabeled_count = len(training_indices) - (len(positives) + len(reliable_negatives))
max_iterations = 10

def stop_criterion(iteration, amount_of_unlabeled_instances):
  global last_iteration_unlabeled_count
  if iteration == max_iterations:
    print('Max iterations reached')
    return True
  if amount_of_unlabeled_instances == 0:
    print('No unlabeled instsances are left')
    return True
  if amount_of_unlabeled_instances == last_iteration_unlabeled_count:
    print('No new reliable negatives were found')
    # jeżeli nie potrafimy już przesunąć granicy decyzyjnej, również kończymy
    return True
  last_iteration_unlabeled_count = amount_of_unlabeled_instances

iteration = 1
while True:
  print(f'Iteration {iteration} started')
  # 'balanced' spowoduje dopasowanie wag do częstości występowania etykiety
  estimator = SVC(class_weight='balanced')
  # przy użyciu stratified shuffle split, losowo wybrane podzbiory walidacji krzyżowej
  # będą posiadały podobny rozkład zmiennej niezależnej
  split = StratifiedShuffleSplit(n_splits=5, train_size=0.9, random_state=9999)
  # wyszukiwanie hiperparametrów
  grid_search = GridSearchCV(
    estimator, params_grid, 
    scoring=make_scorer(f1_score), cv=split, n_jobs=-1, refit=True
  )
  iteration_X = pd.concat((reliable_negatives[features], positives[features]), axis=0)
  iteration_y = pd.concat((reliable_negatives['pu_label'], positives['pu_label']), axis=0)

  # szukanie najlepszego estymatora
  print(f'Training on {len(iteration_y)} observations')
  grid_search.fit(iteration_X, iteration_y)
  best_estimators.append(grid_search.best_estimator_)

  # teraz, należy zaklasfyfikować nieoznaczone obserwacje
  labeled_instances_indices = np.concatenate((reliable_negatives.index, positives.index))
  unlabeled = training_dataset.loc[~training_dataset.index.isin(labeled_instances_indices)]
  prediction = grid_search.best_estimator_.predict(unlabeled[features]).round()

  # rozszerzanie zbioru reliable negatives
  indicated_as_negatives = unlabeled.loc[~prediction.astype(bool)]
  indicated_as_positives = unlabeled.loc[prediction.astype(bool)]
  reliable_negatives = pd.concat((reliable_negatives, indicated_as_negatives), axis=0)
  # do oznaczenia w kolejnej rundzie wybierane są te obserwacje,
  # które model z aktualnej iteracji wskazał jako pozytywne
  unlabeled = indicated_as_positives
  amount_of_unlabeled_instances = len(unlabeled)

  assert len(reliable_negatives) + len(positives) + len(unlabeled) == len(training_indices)
  print(f'Unlabeled instances left: {len(unlabeled)}')

  if stop_criterion(iteration, amount_of_unlabeled_instances):
    print("Stopping")
    break
  iteration += 1

# Klasyfikator zbudowany

Z każdej iteracji zachowałem najlepszy estymator. Dlaczego?

Otóż istnieją różne strategie wyboru ostatecznego modelu. Może to być estymator z najlpeszej rundy, albo ensemble kilku modeli.

Sprawdzimy oba rozwiązania na odłożonym na samym początku zbiorze walidacyjnym. Przypominam, że w zawodowej rzeczywistości moglibyśmy nie posiadać takiej możliwości.

In [0]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix

class VotingClassifier:
  # niestety, implementacja w sklearn wymaga wykonania metody fit()
  # my mamy już wytrenowane estymatory

  def __init__(self, estimators):
    self._estimators = estimators

  def predict(self, X):
    predictions = np.array([clf.predict(X) for clf in self._estimators])
    return np.mean(predictions, axis=0).reshape(-1, ).round()


ensemble = VotingClassifier(best_estimators)
models = {f'Iteration-{i}-model': model for i, model in enumerate(best_estimators)}
models['Ensemble'] = ensemble

for model_id, model in models.items():
  print(f'Evaluating {model_id}')
  predictions = model.predict(test_dataset[features])
  # zbiór walidacyjny był mimo wszystko całkiem zrównoważony
  accuracy_score = balanced_accuracy_score(
      test_dataset['class'],
      predictions
  )
  f1_score_ = f1_score(
      test_dataset['class'],
      predictions
  )
  models[model_id] = {'Model': model_id, 'Accuracy score': accuracy_score, 'F1 score': f1_score_}

pd.DataFrame.from_dict(models.values()).sort_values(['F1 score', 'Accuracy score', 'Model'], ascending=False)

## Podsumowanie

Udało się zbudować całkiem niezły model. To oczywiście nie stanowi gwarancji sukcesu w innych aplikacjach, niejednokrotnie spotkamy się z kiepskimi wynikami.Możemy natrafić na problemy z klasteryzacją, czy zbiór danych, w którym obserwacje obu klas są dość zbliżone do siebie co do rozkładu

## Modyfikacje

Dla chętnych, ciekawym rozwiązaniem jest algorytm GPU (:)) - Generative Positive-Unlabeled. W tym przypadku, zamiast klasteryzować istniejący zbiór danych, proponuje się stworzenie modelu generatywnego klasy pozytywnej. Chodzi o to, by model nauczył się rozkładu atrybutów obserwacji klasy pozytywnej.

Jako zbiór reliable negatives przyjmuje się następnie te obserwacje nieposiadające etykiet, które mają najmniejsze prawdopodobieństwo bycia wygenerowanym przez model generatywny.

Dziękuję!
